In [37]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from sklearn.model_selection import train_test_split

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.13.1


In [90]:
from pymystem3 import Mystem
from tqdm import tqdm_notebook,tqdm_pandas,tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import KeyedVectors

import tf_metrics
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [10]:
tqdm_pandas(tqdm)

### Работа с данными (1 балл)

Загрузите датасет, с которым вы работали во время соревнования на kaggle. Преобразуйте его в формат, удобный для обучения модели. В качетсве фичей используйте эмбединги слов.

In [11]:
mystem = Mystem()

In [12]:
upos_map = {'A':'ADJ','ADV':'ADV','ADVPRO':'ADV','ANUM':'ADJ','APRO':'DET','COM':'ADJ','CONJ':'SCON','INTJ':'INTJ','NONLEX':'X','NUM':'NUM','PART':'PART','PR':'ADP','S':'NOUN','SPRO':'PRON','UNKN':'X' ,'V':'VERB'}

In [13]:
def text_prep_tags(text):
    
    text = text.lower()
    result = []
    # Убираем лишние символы
    #text = re.sub(r'[;,]',r' ',text).strip()
    text = re.sub(r'[^\w\s\.]',r'',text).strip()   
    #text = [token.text for token in razdel.tokenize(text)]
    # Делаем лемматизацию       
#     text = [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace() and lemma not in sw_ru
#             and lemma.strip() not in ['.','..','...']]
    
    
    for item in mystem.analyze(text):
      #  print(item)
        token = None
        if item.get('analysis'):
            lemma = item['analysis'][0]['lex']
            pos = re.split('[=,]', item['analysis'][0]['gr'])[0]
            #and lemma not in sw_ru
            if not lemma.isspace()  and lemma.strip() not in ['.','..','...']: #and lemma not in sw_ru
     
                token = f'{lemma}_{upos_map[pos]}'
        else:
            lem_text = item["text"]
            if not lem_text.isspace() and lem_text.strip() not in ['.','..','...']: #and lem_text not in sw_ru
            
                token = f'{lem_text}_UNKN'
            
        if token:    
            result.append(token)

    
    
    
    # Лемматизатор неправильно разбивает слова с дефисом, поэтому исправляем это
#     if '-' in text:
#         for l in range(len(text)):
#             if text[l] == '-':
#                 text[l] = f'{text[l-1]}-{text[l+1]}'
#                 text[l-1] = text[l+1] = text[l]
    
    return result

In [17]:
X_test = pd.read_csv('test.csv',sep=',')
X_train = pd.read_csv('train.csv',sep=',')

In [15]:
# http://vectors.nlpl.eu/repository/11/185.zip
word2vec = KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [100]:
word2vec.vocab

{'так_ADV': <gensim.models.keyedvectors.Vocab at 0x1c5428d8d0>,
 'мочь_VERB': <gensim.models.keyedvectors.Vocab at 0x1c7a32b358>,
 'быть_VERB': <gensim.models.keyedvectors.Vocab at 0x1c74596128>,
 'человек_NOUN': <gensim.models.keyedvectors.Vocab at 0x1c782d54a8>,
 'xxxxxx_NUM': <gensim.models.keyedvectors.Vocab at 0x1c782d5438>,
 'еще_ADV': <gensim.models.keyedvectors.Vocab at 0x1c782d50b8>,
 'уже_ADV': <gensim.models.keyedvectors.Vocab at 0x1c782d5ac8>,
 'год_NOUN': <gensim.models.keyedvectors.Vocab at 0x1c782d5550>,
 'xxxxxxxx_NUM': <gensim.models.keyedvectors.Vocab at 0x1c782baa90>,
 'один_NUM': <gensim.models.keyedvectors.Vocab at 0x1c782baac8>,
 'сказать_VERB': <gensim.models.keyedvectors.Vocab at 0x1c782ba9b0>,
 'знать_VERB': <gensim.models.keyedvectors.Vocab at 0x1c782dd198>,
 'время_NOUN': <gensim.models.keyedvectors.Vocab at 0x1c782ddbe0>,
 'говорить_VERB': <gensim.models.keyedvectors.Vocab at 0x1c782ddb70>,
 'рука_NOUN': <gensim.models.keyedvectors.Vocab at 0x1c782dd588>,
 '

In [43]:
#X_train['text'] = X_train['text'].progress_apply(text_prep_tags)
#X_test['text'] = X_test['text'].progress_apply(text_prep_tags)
train_ids = X_train['text'].progress_apply(lambda x: list(map(lambda y: count_v.vocabulary_[y] +1 , x)))


100%|██████████| 112467/112467 [00:01<00:00, 68890.13it/s]

In [18]:
count_v = CountVectorizer(tokenizer=text_prep_tags)
count_v.fit(tqdm_notebook(X_train['text']))

In [44]:
X_train_ids,X_val_ids,y_train,y_val = train_test_split(train_ids,X_train['label'],test_size=0.3)

In [46]:
X_train_ids.shape

(78726,)

In [ ]:
len(set(X_train['text'].sum()))

In [51]:
vocab_size = len(count_v.vocabulary_)
sentence_size = 50
embedding_size = 300


# первые индексы в словаре зарезервированы за специальными токенами 
# для паддинга, начала предложения, и для слов которые не вошли в словарь
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2



print("Pad sequences (samples x time)")
X_train_ids = sequence.pad_sequences(X_train_ids, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

print("x_train shape:", X_train_ids.shape)
X_val_ids = sequence.pad_sequences(X_val_ids, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

print("x_train shape:", X_val_ids.shape)


Pad sequences (samples x time)
x_train shape: (78726, 50)
x_train shape: (33741, 50)


In [62]:
def initializer(shape=None, dtype=None, partition_info=None):    
    vocab_dict = count_v.vocabulary_
    embedding_matrix = np.random.uniform(-1, 1, size=(vocab_size+1, embedding_size))
    num_loaded = 0
    for w, i in vocab_dict.items():
        v = None
        try:
            v = word2vec[w]
        except KeyError: # не нашли такой токен в словаре
                pass
        if v is not None :
            embedding_matrix[i+1] = v
            num_loaded += 1
   
    embedding_matrix = embedding_matrix.astype(np.float32)
    return embedding_matrix

In [91]:
x_len_train[np.argmax(x_len_train)]

4194

In [35]:
model_dir = './model'
embedding_size = 300

In [95]:
# column = tf.feature_column.categorical_column_with_identity('x', len(count_vec.vocabulary_)+1)
column = tf.feature_column.embedding_column('x', dimension=300)
#column = tf.feature_column.numeric_column('x')

In [52]:
x_len_train = np.array([len(x) for x in X_train_ids])
x_len_test = np.array([len(x) for x in X_val_ids])

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X_train_ids, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(X_train_ids))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X_val_ids, x_len_test, y_val))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [86]:
all_classifiers = {}
def train_and_evaluate(net,net_name):
    params = {'net':net}
    classifier = tf.estimator.Estimator(model_fn=model_fn,
                                         model_dir=os.path.join(model_dir, net_name), params=params)
    # Save a reference to the classifier to run predictions later
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=15000)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['class_ids'] for p in classifier.predict(input_fn=eval_input_fn)])
        
    # Reset the graph to be able to reuse name scopes
    tf.reset_default_graph() 
    # Add a PR summary in addition to the summaries that the classifier writes
    
    print(tf.metrics.accuracy(y_val,predictions))
#     pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_test.astype(bool), num_thresholds=21)
#     with tf.Session() as sess:
#         writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
#         writer.add_summary(sess.run(pr), global_step=0)
#         writer.close()

In [49]:
# classifier = tf.estimator.DNNClassifier(
#     hidden_units=[100],
#     feature_columns=[column], 
#     model_dir=os.path.join(model_dir, 'bow_embeddings'))
# train_and_evaluate(classifier)

### CNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- Dense

Параметры модели подберите сами.

In [92]:
def model_fn(features, labels, mode, params):  
    
    # Compute predictions.
    net = params['net']
    logits = net(features)
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                               predictions=predicted_classes,
                               name='acc_op')
    f1 = tf_metrics.f1(labels=labels,
                               predictions=predicted_classes,num_classes=3,average='micro')
    
    metrics = {'accuracy': accuracy,'f1':f1}
    tf.summary.scalar('accuracy', accuracy[1])
    
    # Compute evaluation
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)
    
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [73]:
def cnn_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)

    conv = tf.layers.conv1d(
        inputs=inputs,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    #hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    logits = tf.layers.dense(inputs=pool, units=3)

    return logits

In [85]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
reset_graph()

In [83]:

train_and_evaluate(classifier,cnn_fn,'cnn_new')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/cnn_new', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c66774f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was f

INFO:tensorflow:loss = 0.112872794, step = 21601 (1.429 sec)
INFO:tensorflow:global_step/sec: 70.4369
INFO:tensorflow:loss = 0.10627276, step = 21701 (1.420 sec)
INFO:tensorflow:global_step/sec: 70.5969
INFO:tensorflow:loss = 0.17773859, step = 21801 (1.416 sec)
INFO:tensorflow:global_step/sec: 69.7328
INFO:tensorflow:loss = 0.13645457, step = 21901 (1.434 sec)
INFO:tensorflow:global_step/sec: 70.1564
INFO:tensorflow:loss = 0.16961025, step = 22001 (1.425 sec)
INFO:tensorflow:global_step/sec: 65.0814
INFO:tensorflow:loss = 0.12271138, step = 22101 (1.537 sec)
INFO:tensorflow:global_step/sec: 67.2104
INFO:tensorflow:loss = 0.08437645, step = 22201 (1.488 sec)
INFO:tensorflow:global_step/sec: 65.6416
INFO:tensorflow:loss = 0.08549785, step = 22301 (1.524 sec)
INFO:tensorflow:global_step/sec: 69
INFO:tensorflow:loss = 0.1281373, step = 22401 (1.449 sec)
INFO:tensorflow:global_step/sec: 69.1166
INFO:tensorflow:loss = 0.077677295, step = 22501 (1.447 sec)
INFO:tensorflow:global_step/sec: 69

INFO:tensorflow:global_step/sec: 67.8689
INFO:tensorflow:loss = 0.03813668, step = 29801 (1.471 sec)
INFO:tensorflow:global_step/sec: 69.0622
INFO:tensorflow:loss = 0.080762535, step = 29901 (1.448 sec)
INFO:tensorflow:Saving checkpoints for 30000 into ./model/cnn_new/model.ckpt.
INFO:tensorflow:Loss for final step: 0.05464234.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-19T15:11:01Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/cnn_new/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-19-15:11:05
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.8003616, global_step = 30000, loss = 0.91906095
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 30000: ./model/cnn_new/model.ckpt-30000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling mod

### RNN model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- RNN
- Dense

Параметры модели подберите сами.

In [95]:
def rnn_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)

    cell = tf.nn.rnn_cell.BasicRNNCell(num_units=10)
    output, state = tf.nn.dynamic_rnn(cell, inputs,sequence_length=features['len'], dtype=tf.float32)
    
    logits = tf.layers.dense(inputs=state, units=3)

    return logits

In [96]:
train_and_evaluate(rnn_fn,'rnn')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model/rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1cbc881080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was final

INFO:tensorflow:global_step/sec: 72.0219
INFO:tensorflow:loss = 1.0243983, step = 7201 (1.389 sec)
INFO:tensorflow:global_step/sec: 75.1335
INFO:tensorflow:loss = 1.0091954, step = 7301 (1.331 sec)
INFO:tensorflow:global_step/sec: 69.3525
INFO:tensorflow:loss = 1.090903, step = 7401 (1.442 sec)
INFO:tensorflow:global_step/sec: 75.6849
INFO:tensorflow:loss = 1.0127794, step = 7501 (1.322 sec)
INFO:tensorflow:global_step/sec: 78.124
INFO:tensorflow:loss = 1.0826123, step = 7601 (1.280 sec)
INFO:tensorflow:global_step/sec: 69.5629
INFO:tensorflow:loss = 0.9811935, step = 7701 (1.437 sec)
INFO:tensorflow:global_step/sec: 70.946
INFO:tensorflow:loss = 1.0948157, step = 7801 (1.410 sec)
INFO:tensorflow:global_step/sec: 64.5043
INFO:tensorflow:loss = 0.986361, step = 7901 (1.551 sec)
INFO:tensorflow:global_step/sec: 76.0445
INFO:tensorflow:loss = 1.0099255, step = 8001 (1.315 sec)
INFO:tensorflow:global_step/sec: 76.1388
INFO:tensorflow:loss = 1.0439497, step = 8101 (1.313 sec)
INFO:tensorflo

INFO:tensorflow:Finished evaluation at 2019-03-22-14:18:13
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.451676, f1 = 0.451676, global_step = 15000, loss = 1.0399488
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: ./model/rnn/model.ckpt-15000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/rnn/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
(<tf.Tensor 'accuracy/value:0' shape=() dtype=float32>, <tf.Tensor 'accuracy/update_op:0' shape=() dtype=float32>)


### LSTM model (1 балл)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- LSTM
- Dense

Параметры модели подберите сами.

In [ ]:
def lstm_fn(features):
    inputs = tf.contrib.layers.embed_sequence(
        features['x'], vocab_size, embedding_size,
        initializer=initializer, trainable=False)

    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(100)

    # create the complete LSTM
    _, final_states = tf.nn.dynamic_rnn(
        lstm_cell, inputs, sequence_length=features['len'], dtype=tf.float32)

    # get the final hidden states of dimensionality [batch_size x sentence_size]
    outputs = final_states.h
    
    logits = tf.layers.dense(inputs=outputs, units=3)

    return logits

In [ ]:
train_and_evaluate(lstm_fn,'lstm')

Сравните все три реализованные модели по времени и по качеству классификации. Какая лучше? Как думаете, почему?

<b>Вывод:</b>

### Baseline model (2 балла)

Реализуйте модель со следующими слоями, идущими в указанном порядке:
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Параметры для Conv1D, MaxPooling1D и RNN подберите сами.

In [ ]:
# your code here

### Dropout (1 балл)

Добавьте dropout к baseline моделе. Пример модефицированной модели:
- Dropout
- Conv1D, activation='relu'
- MaxPooling1D
- RNN
- Dense

Подберите параметры.

In [ ]:
# your code here

Улучшило ли результат классификации использование Dropout? Как думаете, почему?

<b>Вывод:</b>

### Bidirectional (1 балл)

Вместо RNN в первой модели используйте biRNN. Пример модефецированной модели:
- Conv1D, activation='relu'
- MaxPooling1D
- biRNN
- Dense

Подберите параметры.

In [ ]:
# your code here

Дало ли буст использовании biRNN вместо RNN? Сильный? Как думаете, почему?

<b>Вывод:</b>

### Custom model (2 балла)

Подберите архитектуру сети, используя сверточные слои и слои макспулинга и превзойдите качество полносвязной сети.

Подберите архитектуру сети, используя следующие типы слоев:
- Conv1D
- MaxPooling1D
- RNN
- LSTM
- GRU
- Dropout
- Dense

Можно использовать любые из описанных слоев в любом количестве и в любом порядке. Не обязательно использовать все описанные слои. 

Настройте параметры. Превзойдите лучшее качество, полученное раннее. 

In [ ]:
# your code here